# Multigrid simulation of the Gaussian model

In this exercise we had to implement the multigrid technique to the simulation of the Gaussian model in 1 dimension.

Since we couldn't get our head around the theory, we implemented what we could.

## Expactation values
For the microscopic degree of freedom the real-valued field $u$ with the hamiltonian is given by:
$$H_a(u)=1/a\sum_{k=1}^{N}(u_i-u_{i-1})^2$$

We use the *Dirichlet boundary conditions* for the field $u$:
\begin{align}
    u(0)&=u_0=0\\
    u(L)&=u_N=0
\end{align}

For this problem, the partition sum is given by
$$Z(\beta, N, a)=\prod_{i=1}^{N-1}\int_{-\infty}^\infty du_i \exp(-\beta H_a)$$


With the given fourier decomposition $u_l=\sum_{k=1}^{N-1}c_k\sin(k\pi la/L)$ one can derive the hamiltonian $$H_a(u)=\frac{2N}{a}\sum_{k=1}^{N-1}c_k^2\sin^2\left(\frac{k\pi}{2N}\right)$$

*1. Based on the above Fourier decomposition determine the analytic formula for the
expectation value 1 of*
    
- the magnetization, defined by $m=a/L\sum_{i=1}^{N-1}u_i$
- its square $m^2$ and
- the energy / the Hamiltonian $H_a$

### Magnetization $m$:
The expectation value of the magnetization is:
$$\langle m\rangle=0$$
This is a direct consequence of the missing external magnetic field $h$
### Square of $m$:
                                                        ¯\_(ツ)_/¯
### Energy / Hamiltonian $H_a$:
                                                        ¯\_(ツ)_/¯


## Implementation of the Metropolis-Hastings algorithm
The algorithm works the following way:

1. choose a site $x$ for update , $x$ ∼ $U({1,\dots, N-1})$    
2. propose new $u'(x) = u(x) + r \delta$, with $r$ ∼ $U([−1, 1])$ and $\delta$ a fixed scale parameter
3. Metropolis accept / reject step

*2. Implement the above version of the Metropolis-Hastings sweep. Test your algorithm
by sampling with $\delta= 2.$, $N = 64$, $\beta = 1$. Perform a measurement of the magnetization and
energy after each sweep. Compare to the analytic results for the expectation values.*

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.special as sc
from IPython.display import Markdown, display
delta = 2
N = 64
beta = 1
a = 1
L = a*N
nMeas = 1000
nTherm = 500

def printmd(string): #Method to print fancy markdown text
    display(Markdown(string))

    
def gen_configuration(N): #Method to generate a uniform configuration of size N
    config = np.random.uniform(-10,10,size=N)
    config[0], config[N-1] = 0, 0 # Apply dirichlet boundary conditions 
    return config

def Hamiltonian(config, N, a): #Method to calculate the hamiltonian for a given config
    hamiltonian_sum = 0
    for i in range(1, len(config)):
        hamiltonian_sum += (config[i]-config[i-1])**2
    return 1/a*hamiltonian_sum


def Metropolis(config, delta, beta, a): #Metropolis-Hasting-Algorithm
    for i in range(0, N-1):
        config_update = config.copy()
        x = np.random.randint(1, N-1) #Step 1
        config_update[x]+=np.random.uniform(-1,1)*delta # Step 2
        if np.random.uniform(0,1)<=np.exp(-beta*(Hamiltonian(config_update, N, a)-Hamiltonian(config, N, a))): #Step 3
            config = config_update

    return config
    
def numerical(): #Method to generate the numerical results
    config=gen_configuration(N) #generate config
    m_observed = [] #generate lists to store measurements
    m_squared_observed = []
    energy_observed = []
    for i in range(nTherm): #Do thermalization (if needed)
        config = Metropolis(config, delta, beta, a)
    for i in range(nMeas): #Do nMeas measurements
        config_meas = Metropolis(config, delta, beta, a).copy()
        m_observed.append(a/L*np.sum(config_meas))
        m_squared_observed.append(a/L*np.sum(np.array(config_meas)**2))
        energy_observed.append(Hamiltonian(config_meas, N, a))
    return np.mean(m_observed),np.mean(m_squared_observed), np.mean(energy_observed)

Print out the numerical and analytical results:

In [2]:
m_num, m_squared_num, energy_num = numerical()
printmd("$\langle m_{num} \\rangle $=%.5f, $\langle m_{analytical}\\rangle$=%.5f "%(m_num,0))
printmd("$\langle m^2_{num} \\rangle $=%.5f, $\langle m^2_{analytical}\\rangle$=%s "%(m_squared_num,"¯\_(ツ)_/¯"))
printmd("$\langle E_{num} \\rangle $=%.5f, $\langle E_{analytical}\\rangle$=%s "%(energy_num,"¯\_(ツ)_/¯"))

$\langle m_{num} \rangle $=-0.11087, $\langle m_{analytical}\rangle$=0.00000 

$\langle m^2_{num} \rangle $=1.65959, $\langle m^2_{analytical}\rangle$=¯\_(ツ)_/¯ 

$\langle E_{num} \rangle $=31.13766, $\langle E_{analytical}\rangle$=¯\_(ツ)_/¯ 

The numerical result of $\langle m_{num} \rangle$ is bigger than the analytical value

## Multigrid algorithm

From now on the following Hamiltonian shall be considered:
$$H_a(u)=\frac{1}{a}\sum_{i=1}^N (u_i-u_{i-1})^2+a\sum_{i=1}^{N-1}\phi_i^{(a)}u_i$$
with an external field $\phi^{(a)}$. The prolongation $u^{(a)}=\tilde{u}^{(a)}+I_{(2a)}^{(a)}u^{(2a)}_i$ leads to a decomposition

$$H_a\left(u^{(a)}\right)=H_a\left(\tilde{u}^{(a)}\right)+H_{2a}\left(u^{(2a)}\right)$$
$$H_{2a}\left(u^{(2a)}\right)=\frac{1}{2a}\sum_{i=1}^{N/2}\left(u_i^{(2a)}-u_{i-1}^{(2a)}\right)^2+2a\sum_{i=1}^{N/2-1}\phi_i^{(2a)}u_i^{(2a)}$$

*3. Give the explicit form of $\phi^{(2a)}$: how does the coarse-level external field $\phi^{(2a)}$ depend
on the fine-level fields ? Implement the restriction and prolongation functions.*

The explicit form of $\phi^{(2a)}$ is given as:
$$\phi^{(2a)}=\frac{1}{4}\left(\phi_{2i-1}^{(a)}+2\phi_{2i}^{(a)}+\phi_{2i+1}^{(a)}\right)$$

### Implementation of the restriction and prolongation functions

In [3]:
config = gen_configuration(N+1)
def fine_to_coarse(config): #restriction function
     return [config[i] for i in range(len(config)) if i%2==0]

def linear_interpolation(config_coarse): #Method for linear interpolation
    linear_interpolation_list = []
    for i in range(len(config_coarse)):
        linear_interpolation_list.append(config_coarse[i])
        if i < len(config_coarse)-1: 
            linear_interpolation_list.append((config_coarse[i-1]+config_coarse[i+1])/2)
    return linear_interpolation_list
    
def coarse_to_fine(config, config_coarse): #prolongation function
    return np.array(config)+np.array(linear_interpolation(config_coarse))

In [4]:
config_coarse = fine_to_coarse(config)
config_interpol = coarse_to_fine(config, config_coarse)